<a href="https://colab.research.google.com/github/dream80/DFLWorkspace/blob/master/DeepFaceLab_Colab_V5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 简介


这是一个在线AI换脸的脚本，只需要用浏览器打开，点击相应的步骤即可完成视频换脸。具体操作参考视频教程。

此脚本由**[托尼不是塔克](https://www.tonyisstark.com)**创建，仅用于学习，请勿滥用。如有疑问，拉到最后！

视频教程：https://www.youtube.com/watch?v=MgcnGnfVDcc&list=PL4wPj6o65_XHcqBZ7iMVEaKrPi5-LrYNZ

In [ ]:
#@title 查看分配到的设备
# 查看设备，是K80还是T4,如果是K80...！
! /opt/bin/nvidia-smi

# 第一步 准备工作






注意：  
初始化项目root指定你自己路径，一般  
云端硬盘路径为：/content/drive/MyDrive/  
共享云端路径为：/content/drive/Shareddrives/DriverName  
DriverName：共享盘名称，每个人不一样，根据自己情况来改。

*准备素材不要乱点，在已经有素材和模型的情况下！！！

In [ ]:
#@title 挂载云盘
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title 初始化项目

import os
root = "/content/drive/MyDrive/" #@param {type:"string"}
%cd $root
!mkdir DeepFaceLab
deepfacelab=os.path.join(root,"DeepFaceLab")
workspace=os.path.join(deepfacelab,"workspace")
deepfacelab_cloab=os.path.join(deepfacelab,"DeepFaceLab_Colab")



In [ ]:
#@title 准备素材（点击后会清空workspace）

online = True #@param {type:"boolean"}
upload = "data_src/aligned" #@param ["data_src/aligned","data_dst/aligned","model","data_src","data_dst"]

if online :
  %cd $deepfacelab
  !rm -rf workspace
  !git clone https://github.com/dream80/DFLWorkspace.git workspace
else:
  from google.colab import files
  import shutil
  upload_path = os.path.join(workspace,upload)
  %cd $upload_path
  #if os.path.isdir(upload_path):
  #shutil.rmtree(upload_path)
  uploaded = files.upload()
  '''for filename in uploaded.keys():
    print(basepath+"/"+filename)
    print(upload_path+"/"+filename)
    shutil.move(os.path.join(basepath, filename), os.path.join(upload_path, filename))
  '''


# 第二部 安装软件

In [ ]:
#@title 开始安装

Version = "v3.07.20.1" #@param ["v2.02.03", "v2.02.23b", "v2.02.28","v2.03.07","v2.03.15","v2.06.01","v2.06.19","v2.08.02","v3.01.04","v3.07.20","v3.07.20.1","last"]
%cd $deepfacelab
!rm -fr DeepFaceLab_Colab

if Version=="620":
  print("620版加载中....")
  # 获取DFL源代码v1.6.1稳定版
  !git clone -b v1.6.1 https://github.com/dream80/DeepFaceLab_Colab.git
  %cd $deepfacelab_cloab
  !pip install -r requirements-colab.txt
elif Version=="last":
  print("最新版加载中....")
  !git clone https://github.com/iperov/DeepFaceLab.git DeepFaceLab_Colab
  %cd $deepfacelab_cloab
  !pip install h5py
  !pip install opencv-python
  !pip install ffmpeg-python==0.1.17
  !pip install colorama
  !pip install tf2onnx==1.9.3
else:
  print(Version+" 加载中....")
  cmd="clone -b "+Version+" https://github.com/dream80/DeepFaceLab_Colab.git"
  !git $cmd
  %cd $deepfacelab_cloab
  !pip install h5py
  !pip install opencv-python
  !pip install ffmpeg-python==0.1.17
  !pip install colorama
  !pip install tf2onnx==1.9.3

!python -m pip install numpy==1.23.5


# 第三步. 提取脸部

In [ ]:
#@title 开始提取SRC

%cd $deepfacelab_cloab
print("\n===分解视频==================================\n")
!python main.py videoed extract-video --input-file ../workspace/data_src.mp4 --output-dir ../workspace/data_src/
print("\n===提取人脸==================================\n")
!python main.py extract --input-dir ../workspace/data_src --output-dir ../workspace/data_src/aligned --detector s3fd
print("\n===打包人脸==================================\n")
!python main.py util --input-dir ../workspace/data_src/aligned  --pack-faceset


In [ ]:
#@title 开始提取DST
%cd $deepfacelab_cloab
print("\n===分解视频==================================\n")
!python main.py videoed extract-video --input-file ../workspace/data_dst.mp4 --output-dir ../workspace/data_dst/
print("\n===提取人脸==================================\n")
!python main.py extract --input-dir ../workspace/data_dst --output-dir ../workspace/data_dst/aligned --detector s3fd --output-debug
print("\n===打包人脸==================================\n")
!python main.py util --input-dir ../workspace/data_dst/aligned  --pack-faceset


# 第四步. 训练模型

In [ ]:
#@title 训练模型
Model = "SAEHD" #@param ["SAEHD","AMP","Quick96","XSeg"]
%cd $deepfacelab_cloab
cmd = "main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model "+Model+" --no-preview"
!python $cmd

In [ ]:
#@title SRC-SRC
Model = "SAEHD" #@param ["SAEHD","AMP","Quick96","XSeg"]
%cd $deepfacelab_cloab
cmd = "main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_src/aligned --model-dir ../workspace/model --model "+Model+" --no-preview"
!python $cmd

# 第五步. 转换输出



In [ ]:
#@title 应用模型

Model = "SAEHD" #@param ["SAEHD", "Quick96"]
%cd $deepfacelab_cloab
cmd = " main.py merge --output-mask-dir ../workspace/data_dst/merged_mask --input-dir ../workspace/data_dst --output-dir ../workspace/data_dst/merged --aligned-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model " + Model
!python $cmd

In [ ]:
#@title 合成视频
%cd $deepfacelab_cloab

format = "mp4" #@param ["mp4","avi","mov"]
lossless = False #@param {type:"boolean"}

merged = "main.py videoed video-from-sequence --input-dir ../workspace/data_dst/merged --output-file ../workspace/result.mp4 --reference-file ../workspace/data_dst.mp4 --include-audio"
merged_mask = "main.py videoed video-from-sequence --input-dir ../workspace/data_dst/merged_mask --output-file ../workspace/result_mask.mp4 --reference-file ../workspace/data_dst.mp4 --include-audio --lossless"


if lossless :
  merged=merged+" --include-audio --lossless"

!python $merged
!python $merged_mask



#第六步. 继续训练



In [ ]:
#@title 一键运行
#挂载谷歌云盘
#点击链接授权，复制授权码，填入框框，然后回车。

Model = "SAEHD" #@param ["SAEHD","AMP","Quick96","XSeg"]

#挂载
from google.colab import drive
drive.mount('/content/drive')

import os
root = "/content/drive/MyDrive/" #@param {type:"string"}

%cd $root
!mkdir DeepFaceLab
deepfacelab=os.path.join(root,"DeepFaceLab")
workspace=os.path.join(deepfacelab,"workspace")
deepfacelab_cloab=os.path.join(deepfacelab,"DeepFaceLab_Colab")

%cd $deepfacelab_cloab
# 安装Python依赖
!pip install h5py
!pip install opencv-python
!pip install ffmpeg-python==0.1.17
!pip install colorama
!pip install tf2onnx==1.9.3
# 开始训练SAE ，如果是其他模型，修改后面的参数即可，比如H128。
cmd = "main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model "+Model+" --no-preview"
!python $cmd

# 工具



打包素材。可以加快上传，下载，加载的速度。

In [ ]:
#@title 开始排序
target = "src" #@param ["src","dst"]

if target=="src":
  #Src排序，可以通过谷歌云盘查看结果，删除不良图片
  cmd = "main.py sort --input-dir ../workspace/data_src/aligned"

else:
  cmd = "main.py sort --input-dir ../workspace/data_dst/aligned "

!python $cmd

In [ ]:
#@title 素材打包

target = "dst" #@param ["src","dst"]
%cd $deepfacelab_cloab
if target=="src":
  !python main.py util --input-dir ../workspace/data_src/aligned  --pack-faceset
else:
  !python main.py util --input-dir ../workspace/data_dst/aligned  --pack-faceset



In [ ]:
#@title 素材解包

target = "src" #@param ["src","dst"]
%cd $deepfacelab_cloab
if target=="src":
  !python main.py util --input-dir ../workspace/data_src/aligned --unpack-faceset
else:
  !python main.py util --input-dir ../workspace/data_dst/aligned --unpack-faceset



In [ ]:
#@title 素材增强

target = "src" #@param ["src"]
%cd $deepfacelab_cloab
if target=="src":
  !python main.py facesettool enhance --input-dir ../workspace/data_src/aligned
else:
  !python main.py facesettool enhance --input-dir ../workspace/data_dst/aligned



# 其他


 谷歌云地址：https://drive.google.com/drive/my-drive

 作者邮箱 ：wpgdream@gmail.com

 知识星球：https://t.zsxq.com/vFuzRNr

 Github ：https://github.com/dream80/DeepFaceLab_Colab  
 有用就点个star 谢谢!!!  
  
